In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

### Parse

In [2]:
# try:
#     os.remove('snorkel.db')
# except:
#     pass

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [3]:
# import os
# from snorkel.parser import CorpusParser, HTMLParser, OmniParser
# from snorkel.utils import get_ORM_instance
# from snorkel.queries import split_corpus

# docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/html/'
# doc_parser = HTMLParser(path=docs_path)
# context_parser = OmniParser(blacklist=['style'], 
#                             flatten=['span','br'], flatten_delim='',
#                             tabular=True, visual=False, lingual=True)
# cp = CorpusParser(doc_parser, context_parser, max_docs=125)

# %time corpus = cp.parse_corpus(name='Hardware', session=session)

# session.add(corpus)
# session.commit()

In [4]:
# If necessary:
# import os
# os.system('cp snorkel.db snorkel.db\ corpus');

### Load

In [5]:
from snorkel.models import Corpus
import os

os.system('cp snorkel.db\ corpus snorkel.db');
from snorkel import SnorkelSession
session = SnorkelSession()

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware) contains 122 Documents


### 1st pass

In [6]:
# Make parts list
from hardware_utils import get_gold_dict
from collections import defaultdict
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/hardware_dev_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
parts_by_doc = defaultdict(set)
for part in gold_parts:
    parts_by_doc[part[0]].add(part[1])

### Extraction

In [7]:
from hardware_utils import OmniNgramsPart, load_extended_parts_dict, part_matcher
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])
part_ngrams = OmniNgramsPart(n_max=3, parts_by_doc=parts_by_doc)
part_matcher = part_matcher()

# from snorkel.matchers import RegexMatchSpan, Inverse, Intersect
# long_enough = RegexMatchSpan(rgx='[A-Z1-2][A-Z0-9\-]{4,}(?:\/(?:DG|HF))?', ignore_case=False)
# at_least_one_number = RegexMatchSpan(rgx='[A-Z0-9\-\/]*[0-9][A-Z0-9\-\/]*', ignore_case=False)
# at_least_one_letter = RegexMatchSpan(rgx='[A-Z0-9\-\/]*[A-Z][A-Z0-9\-\/]*', ignore_case=False) 
# not_case_number = Inverse(RegexMatchSpan(rgx='TO-\d+'))
# part_matcher = Intersect(long_enough, at_least_one_number, at_least_one_letter, not_case_number)

In [8]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, 
                        [part_ngrams], 
                        [part_matcher],
                        throttler=None)

In [9]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%
CPU times: user 52.3 s, sys: 831 ms, total: 53.2 s
Wall time: 53.8 s
Candidate Set (Hardware Candidates) contains 13100 Candidates


In [10]:
# eeca_rgx = '([ABC][A-Z][WXYZ]?[0-9]{3,5}(?:[A-Z]){0,5}[0-9]?[A-Z]?(?:-[A-Z0-9]{1,7})?(?:[-][A-Z0-9]{1,2})?)'
# jedec_rgx = '([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)'
# print '|'.join([eeca_rgx, jedec_rgx])

### Evaluation

In [11]:
from hardware_utils import get_gold_dict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/hardware_dev_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
print "%d gold parts loaded" % len(gold_parts)

949 gold parts loaded


In [12]:
from hardware_utils import parts_f1

%time tp, fp, fn = parts_f1(candidates, gold_parts)

Scoring on Entity-Level Gold Data
Corpus Precision 0.607
Corpus Recall    0.927
Corpus F1        0.734
----------------------------------------
TP: 880 | FP: 569 | FN: 69

CPU times: user 46.6 s, sys: 1.14 s, total: 47.7 s
Wall time: 48.3 s


In [13]:
from pprint import pprint
pprint(sorted(fn)[:])

[('CENTRALSEMICONDUCTORCORP_CENU45', 'CEN-U45'),
 ('CENTRALSEMICONDUCTORCORP_CMPT5401ETR', 'CMPT5401E'),
 ('CENTRALSEMICONDUCTORCORP_CXT4033TR', 'CXT4033'),
 ('DIODESINCORPORATED_2DD26527', '2DD2652'),
 ('DIODESINCORPORATED_2DD26527', '2DD2652-7'),
 ('DIODESINCORPORATED_FCX491ATA', 'FCX491A'),
 ('DIODESINCORPORATED_ZXT690BKTC', 'ZXT690BKQTC'),
 ('DIODESINCORPORATED_ZXT690BKTC', 'ZXT690BKTC'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310O'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310R'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310Y'),
 ('MICROCOMMERCIALCO_2N3904AP', '2N3904-AP'),
 ('MICROCOMMERCIALCO_2N3904AP', '2N3904-AP-HF'),
 ('MICROCOMMERCIALCO_2N3904AP', '2N3904-BP'),
 ('MICROCOMMERCIALCO_2N3904AP', '2N3904-BP-HF'),
 ('MICROCOMMERCIALCO_TIP29ABP', 'TIP29A-BP-HF'),
 ('MICROCOMMERCIALCO_TIP29ABP', 'TIP29B-BP-HF'),
 ('MICROCOMMERCIALCO_TIP29ABP', 'TIP29C-BP-HF'),
 ('MICROCOMMERCIALCO_TIP29ABP', 'TIP30A-BP-HF'),
 ('MICROCOM

The End.